In [1]:
from selenium import webdriver
import time 
import pandas as pd
import numpy as np 
import re 

### Use Selenium to Open URL

In [2]:
driver = webdriver.Chrome("C:\chromedriver_win32\chromedriver.exe")
url = 'https://acis.aphis.edc.usda.gov/ords/f?p=118:205'
driver.get(url)

### Use Selenium to Click Table Then Itterate through the pages to add Each Element to a List

In [3]:
xpath= '//*[@id="report_R81916045860772380"]/tbody[2]/tr/td/table'
next_button = '//*[@id="report_R81916045860772380"]/tbody[3]/tr/td/table/tbody/tr/td[4]/a'
counter = 0
lst = []

driver.find_element_by_xpath('//*[@id="P205_ARR_INFO_B"]/span').click()

while counter <= 458:
    time.sleep(5)
    driver.find_element_by_xpath(xpath)
    for tr in driver.find_elements_by_xpath(xpath):
        tds = tr.find_elements_by_tag_name('td')
        lst.append([td.text.replace('\n', ' ').strip() for td in tds])
    driver.find_element_by_xpath(next_button).click()
    time.sleep(2)
    counter += 1 

In [4]:
print(lst)

[['', '-', '-', 'Customer No:    505775 Certificate No:   96-R-0007', '2019', 'Dogs    0   Non-Human Primates   0 Cats    0   Sheep     0 Guinea Pigs   0   Pig     0 Hamsters   0   Other Farm Animals   0 Rabbits   0   All Other Animals   4', '', '-', '-', 'Customer No:    610 Certificate No:   35-R-0135', '2019', 'Dogs    0   Non-Human Primates   0 Cats    0   Sheep     0 Guinea Pigs   0   Pig     0 Hamsters   24   Other Farm Animals   0 Rabbits   0   All Other Animals   0', '', '-', '-', 'Customer No:    83 Certificate No:   51-R-0102', '2019', 'Dogs    0   Non-Human Primates   0 Cats    0   Sheep     0 Guinea Pigs   0   Pig     0 Hamsters   0   Other Farm Animals   0 Rabbits   0   All Other Animals   110', '', '-', '-', 'Customer No:    505884 Certificate No:   73-R-0123', '2019', 'Dogs    0   Non-Human Primates   0 Cats    0   Sheep     0 Guinea Pigs   0   Pig     0 Hamsters   0   Other Farm Animals   0 Rabbits   40   All Other Animals   0', '', '-', '-', 'Customer No:    506106 Cer

### Reshape List into DF 

In [5]:
df = pd.DataFrame(np.array(lst).reshape(-1 ,6))

### Rename Columns

In [6]:
df.columns = ['view_report',
              'exception', 
              'col_e_narrative',
              'registrant_certificates',
              'year',
              'animals_information']

### Convert Year Column to INT

In [7]:
df['year'] = pd.to_numeric(df['year'], errors='coerce')

### Create Certificate Number and Customer Number Column 

In [9]:
df['certificate_num'] = df['registrant_certificates'].apply(lambda x:x.split('Certificate No:')[1])
df['customer_number'] = df['registrant_certificates'].str.extract('(\d+)')

### Create Function to Seperate Numbers in a Column into a Series

In [10]:
number_finder = lambda s: [int(s) for s in str.split(s) if s.isdigit()]

### Apply Number Finder Function to 'animals_information' Column and create new column 'animals'

In [11]:
df['animals'] = df['animals_information'].apply(number_finder)

### Split List from 'animals' column into New Rows

In [12]:
df[['dogs',
    'non-human_primates',
    'cats',
    'sheep',
    'guinea_pigs',
    'pig',
    'hamsters',
    'other_farm_animals',
    'rabbits',
    'all_other_animals']] = pd.DataFrame(df.animals.tolist(), index= df.index)

### Drop Un-needed Columns

In [13]:
df = df.drop(columns = ['registrant_certificates',
                        'animals_information',
                        'animals', 'view_report',
                        'exception', 'col_e_narrative'])

### Verify Data is Correct

In [14]:
df.tail()

,year,certificate_num,customer_number,dogs,non-human_primates,cats,sheep,guinea_pigs,pig,hamsters,other_farm_animals,rabbits,all_other_animals
6880,2014,32-R-0041,17782,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6881,2014,84-R-0065,18462,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6882,2014,93-R-0467,18788,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6883,2014,31-R-0116,18920,0,59,0.0,7.0,0.0,11.0,0.0,0.0,7.0,176.0
6884,2014,41-R-0063,19782,0,0,0.0,0.0,6.0,0.0,0.0,0.0,2.0,0.0


In [15]:
df.to_excel(r'C:\Users\harri\OneDrive\Documents\USDA Reports\Extracted_USDA_Report.xlsx', index = False)